In [1]:
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
#import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from nltk.stem import PorterStemmer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from sklearn import metrics
import string
from sklearn.metrics import precision_recall_fscore_support as score
from xgboost import XGBClassifier


#df = pd.DataFrame()


files = pd.read_csv('Womens Clothing E-Commerce Reviews.csv',error_bad_lines=False,
                    warn_bad_lines=False,
                    low_memory=False)
#files.head()

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 100

np.set_printoptions(threshold=np.inf)

In [2]:
import nltk  
from nltk.tokenize import WordPunctTokenizer

In [3]:
#to lower case
files['Review_Text'] = files.Review_Text.str.lower() 

files

,Unnamed: 0,Clothing ID,Age,Title,Review_Text,Rating,Recommended_IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,i had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"i love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,this shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,5,1080,49,Not for the very petite,"i love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
6,6,858,39,Cagrcoal shimmer fun,i aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits
7,7,858,39,"Shimmer, surprisingly goes with lots","i ordered this in carbon for store pick up, an...",4,1,4,General Petite,Tops,Knits
8,8,1077,24,Flattering,i love this dress. i usually get an xs but it ...,5,1,0,General,Dresses,Dresses
9,9,1077,34,Such a fun dress!,"i'm 5""5' and 125 lbs. i ordered the s petite t...",5,1,0,General,Dresses,Dresses


In [4]:
#Punctuations
files['Review_Text'] = files.Review_Text.str.replace('[^\w\s]', '')
files

,Unnamed: 0,Clothing ID,Age,Title,Review_Text,Rating,Recommended_IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,absolutely wonderful silky and sexy and comfo...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,love this dress its sooo pretty i happened t...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,i had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,i love love love this jumpsuit its fun flirty ...,5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,this shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,5,1080,49,Not for the very petite,i love tracy reese dresses but this one is not...,2,0,4,General,Dresses,Dresses
6,6,858,39,Cagrcoal shimmer fun,i aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits
7,7,858,39,"Shimmer, surprisingly goes with lots",i ordered this in carbon for store pick up and...,4,1,4,General Petite,Tops,Knits
8,8,1077,24,Flattering,i love this dress i usually get an xs but it r...,5,1,0,General,Dresses,Dresses
9,9,1077,34,Such a fun dress!,im 55 and 125 lbs i ordered the s petite to ma...,5,1,0,General,Dresses,Dresses


In [5]:
#tokenize into single words
from nltk.tokenize import word_tokenize
files['Review_Text'].dropna(inplace=True)
files['Review_Text'] = files['Review_Text'].apply(word_tokenize)

In [6]:
#drop NA Review Text
files = files.dropna(how='any',axis=0) 

In [7]:
#stemming
stemmer = PorterStemmer()

files['Review_Text'].apply(lambda x: [stemmer.stem(y) for y in x])
files['Review_Text'] = files['Review_Text'].apply(lambda x: [stemmer.stem(y) for y in x])
files

,Unnamed: 0,Clothing ID,Age,Title,Review_Text,Rating,Recommended_IND,Positive Feedback Count,Division Name,Department Name,Class Name
2,2,1077,60,Some major design flaws,"[i, had, such, high, hope, for, thi, dress, an...",3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"[i, love, love, love, thi, jumpsuit, it, fun, ...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,"[thi, shirt, is, veri, flatter, to, all, due, ...",5,1,6,General,Tops,Blouses
5,5,1080,49,Not for the very petite,"[i, love, traci, rees, dress, but, thi, one, i...",2,0,4,General,Dresses,Dresses
6,6,858,39,Cagrcoal shimmer fun,"[i, ade, thi, in, my, basket, at, hte, last, m...",5,1,1,General Petite,Tops,Knits
7,7,858,39,"Shimmer, surprisingly goes with lots","[i, order, thi, in, carbon, for, store, pick, ...",4,1,4,General Petite,Tops,Knits
8,8,1077,24,Flattering,"[i, love, thi, dress, i, usual, get, an, xs, b...",5,1,0,General,Dresses,Dresses
9,9,1077,34,Such a fun dress!,"[im, 55, and, 125, lb, i, order, the, s, petit...",5,1,0,General,Dresses,Dresses
10,10,1077,53,Dress looks like it's made of cheap material,"[dress, run, small, esp, where, the, zipper, a...",3,0,14,General,Dresses,Dresses
12,12,1095,53,Perfect!!!,"[more, and, more, i, find, myself, reliant, on...",5,1,2,General Petite,Dresses,Dresses


In [8]:
#tfidf_vect = TfidfVectorizer(analyzer = 'word')
#X_train_vect = tfidf_vect.fit_transform(X_train)
#X_val_vect = tfidf_vect.transform(X_val)
#y_train1_vect = tfidf_vect.fit_transform(y_train1)
#y_test1_vect = tfidf_vect.transform(y_test1)


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# This converts the list of words into space-separated strings
files['Review_Text'] = files['Review_Text'].apply(lambda x: ' '.join(x))

count_vect = CountVectorizer()  
counts = count_vect.fit_transform(files['Review_Text'])  

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer().fit(counts)

counts = transformer.transform(counts)  

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(counts,
                                                 files["Recommended_IND"],
                                                 test_size=0.2,
                                                 random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [13]:
#X_train
#tfidf_vect = TfidfVectorizer(analyzer = 'word')
#X_train_vect = tfidf_vect.fit_transform(X_train)
#X_val_vect = tfidf_vect.transform(X_val)
#y_train1_vect = tfidf_vect.fit_transform(y_train1)
#y_test1_vect = tfidf_vect.transform(y_test1)

In [14]:
#GaussianNB()
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train, y_train)

predicted = model.predict(X_val)

print(np.mean(predicted == y_val))  

0.8171878972794304


In [15]:
model = XGBClassifier()
model.fit(X_train, y_train)
predicted = model.predict(X_val)
print(np.mean(predicted == y_val))  



0.8563437579455886


C:\Users\ander\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [16]:
#from keras.models import Sequential

#model = Sequential()

In [17]:
#from keras.layers import Dense

#model.add(Dense(units=64, activation='relu', input_dim=14098))
#model.add(Dense(units=10, activation='softmax'))

#model.compile(loss='categorical_crossentropy',
#              optimizer='sgd',
#              metrics=['accuracy'])


#model.fit(X_train, y_train, epochs=105, batch_size=32)
#scores = model.evaluate(X_train, y_train)
#print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [18]:
X_train

<11796x14098 sparse matrix of type '<class 'numpy.float64'>'
	with 512026 stored elements in Compressed Sparse Row format>

In [19]:
#y_train
y_val

22104    0
19704    1
14557    1
188      1
15561    1
9243     1
10792    1
13556    1
7576     0
9228     1
8324     0
3964     1
11182    1
20668    1
21716    1
17253    1
19055    1
7321     1
15362    1
821      1
17740    1
448      1
9453     0
9557     1
1153     1
12645    1
1391     0
14349    1
14813    1
19325    1
        ..
360      1
16294    1
2120     1
16387    1
6703     1
1925     1
655      1
1172     1
15046    1
21429    1
8968     1
19275    0
1033     1
7165     0
12510    0
262      0
1240     0
15543    1
14666    1
6748     1
13137    1
12743    0
1606     1
2569     1
9867     0
6299     1
1255     1
4649     1
9537     1
9821     1
Name: Recommended_IND, Length: 3933, dtype: int64

In [20]:
#tfidf_vect = TfidfVectorizer(analyzer = 'word')
#X_train_vect = tfidf_vect.fit_transform(X_train)
#X_val_vect = tfidf_vect.transform(X_val)
#y_train1_vect = tfidf_vect.fit_transform(y_train1)
#y_test1_vect = tfidf_vect.transform(y_test1)

In [22]:
from sklearn.feature_selection import SelectKBest, chi2

ch2_result = []
for n in np.arange(1000,8000,1000):
    ch2 = SelectKBest(chi2, k=n)
    x_train_chi2_selected = ch2.fit_transform(X_train, y_train)
    x_validation_chi2_selected = ch2.transform(X_val)
    clf = MultinomialNB()
    clf.fit(x_train_chi2_selected, y_train)
    score = clf.score(x_validation_chi2_selected, y_val)
    ch2_result.append(score)
    #print "chi2 feature selection evaluation calculated for {} features".format(n)

ch2_result



[0.8263412153572337,
 0.8227815916603102,
 0.8227815916603102,
 0.8215102974828375,
 0.8199847444698704,
 0.8187134502923976,
 0.8182049326214086]

In [23]:
ch2_result = []
for n in np.arange(1000,8000,1000):
    ch2 = SelectKBest(chi2, k=n)
    x_train_chi2_selected = ch2.fit_transform(X_train, y_train)
    x_validation_chi2_selected = ch2.transform(X_val)
    clf = LogisticRegression()
    clf.fit(x_train_chi2_selected, y_train)
    score = clf.score(x_validation_chi2_selected, y_val)
    ch2_result.append(score)
    #print "chi2 feature selection evaluation calculated for {} features".format(n)

ch2_result


[0.8766844647851513,
 0.8779557589626239,
 0.881769641495042,
 0.8830409356725146,
 0.882278159166031,
 0.8825324180015256,
 0.8840579710144928]